<a href="https://colab.research.google.com/github/hikmahealth/covid19countymap/blob/master/Hikma_COVID_19_Native_Community_Data_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hikma COVID-19 Response - Native Community Google Sheet Export

License: Apache 2.0

In [ ]:
! pip install fuzzywuzzy
! pip install python-Levenshtein



     |████████████████████████████████| 51kB 1.5MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144803 sha256=c1b3ca855ce52d7ecc1cabade7877e29d5a5bdf8588b049eefcb5e22bcd034ed
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import copy
import io
import json
import urllib

from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
import pandas as pd
import numpy as np

In [ ]:
with urllib.request.urlopen("https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv") as infile:
  raw_data = infile.read()
# Deal with encoding issues in the Census CSV file.
replacements = {0xed: "í", 0xe1: "á", 0xf3: "ó", 0xf1: "ñ", 0xfc:"ü"}
for char, repl in replacements.items():
  raw_data = raw_data.replace(bytes([char]), repl.encode())
raw_census = pd.read_csv(io.BytesIO(raw_data))

## Use census data to map county name to AIAN 2010 census code

In [ ]:
# # Import AIAN 2010 Census Codes wherever available for Native Communities
with urllib.request.urlopen("https://www2.census.gov/geo/docs/reference/codes/AIAlist.txt") as infile:
  raw_data = infile.read()
# Deal with encoding issues in the Census CSV file.
replacements = {0xed: "í", 0xe1: "á", 0xf3: "ó", 0xf1: "ñ", 0xfc:"ü"}
for char, repl in replacements.items():
  raw_data = raw_data.replace(bytes([char]), repl.encode())

# raw_data = raw_data.replace(bytes('\r'), bytes())
# Need to handle names with commas i.e. "Coos, Lower Umpqua, and Siuslaw Reservation and Off-Reservation Trust Land"
searchstr = ', '
replacestr = '; '
raw_data = raw_data.replace(searchstr.encode(), replacestr.encode())
raw_census = pd.read_csv(io.BytesIO(raw_data))



In [ ]:
# Restoring name after parsing
raw_census['AIANNHNAME'] = raw_census['AIANNHNAME'].str.replace('; ', ', ')
raw_census.head()

,AIANNHCE,AIANNHNAME
0,10,Acoma Pueblo and Off-Reservation Trust Land
1,20,Agua Caliente Indian Reservation and Off-Reser...
2,50,Alabama-Coushatta Reservation and Off-Reservat...
3,80,Allegany Reservation
4,95,Alturas Indian Rancheria


In [ ]:
census = raw_census.copy()
census = census.set_index("AIANNHCE")
aian_code_mapping = {name: i for i, name in census.AIANNHNAME.items()}

In [ ]:
with open('testcensus.csv', 'w') as outfile:
  outfile.write(raw_census.to_csv())

## Read the data from Google Sheets

In [ ]:
auth.authenticate_user()
sheets_client = gspread.authorize(GoogleCredentials.get_application_default())
sheet = sheets_client.open_by_url("https://docs.google.com/spreadsheets/d/1r7WLefBmWzAaGgaA9c89t6hampOuKkyfz2Hll-Iq0uY/edit#gid=1147188869")

In [ ]:
raw_policies = pd.DataFrame(sheet.sheet1.get_all_records())

## Clean up the county-level data

### Encode free-text column values

In [ ]:
dimensions = ["school", "work", "shelter", "shelter_enforcement", "event", "testing", "transport", "revenue", "checkpoint"]

In [ ]:
full_policies = raw_policies.rename(columns={
    "You are filling out the form for the following community:": "community_name",
    "What is the population of the community?": "population",
    "Are all schools closed in this community?": "school",
    "If yes, please provide the URL to information regarding school closures.": "school_url",
    "If yes, please provide the date of when this policy on school closures took or will take effect.": "school_date",
    "Are non-essential workplaces closed? Is there any language that suggests that those who do not provide essential services should work from home or stop working?": "work",
    "If yes, please provide the URL to information regarding non-essential workplace closures.": "work_url",
    "If yes, provide the date of when this policy took or will take effect.": "work_date",
    "Is social distancing or shelter-in-place/curfew being enforced with fines or penalties in this community? ": "shelter_enforcement",
    "If yes, please provide the URL to information regarding fine or penalty enforcement.": "shelter_enforcement_url",
    "If yes, please provide the date of when this fine or penalty enforcement took or will take effect.": "shelter_enforcement_date",
    "Is there a shelter-in-place, stay-at-home, or curfew order in this community? For example, were people advised to maintain social distance or abstain from all physical interactions or non-essential trips whenever possible?": "shelter",
    "If yes, please provide the URL to information regarding the shelter-in-place/curfew or social distancing policy.": "shelter_url",
    "If yes, please provide the date of when this shelter-in-place/curfew or social distancing policy took or will take effect.": "shelter_date",
    "Is there any order that public events should be restricted in size or outright cancelled?": "event",
    "If yes, please provide the URL to information regarding public events restrictions.": "event_url",
    "If yes, please provide the date of when the cancellation of public events was or will be in effect.": "event_date",
    "Are there public facilities with COVID testing available in this community? ": "testing",
    "If yes, please provide the URL to information regarding testing facilities.": "testing_url",
    "If yes, please provide the date of when these facilities became available.": "testing_date",
    "If yes, what is the one-way travel time (in minutes) from the geographic center of this community to the nearest COVID testing facility?": "testing_time",
    "Look for any information about whether this community's buses, subways, light rail, etc. are being shut down. Is any public transit system shut down?": "transport",
    "If yes, please provide the URL to information regarding public transit.": "transport_url",
    "If yes, please provide the date of when public transit was or will be stopped.": "transport_date",
    "Does this community participate in the Food Distribution Program on Indian Reservations (FDPIR)?": "fdpir",
    "If yes, please provide the URL to information regarding the Food Distribution Program on Indian Reservations (FDPIR).": "fdpir_url",
    "Are casinos, resorts or other community-owned revenue-generating facilities closed in this community?": "revenue",
    "If yes, please provide the date when revenue-generating facilities closed.": "revenue_date",
    "If yes, please provide the URL to information regarding these facility closures.": "revenue_url",
    "Are there checkpoints in place to regulate traffic through the community?": "checkpoint",
    "If yes, please provide the date of when these checkpoints were implemented.": "checkpoint_date",
    "If yes, please provide the URL to information regarding checkpoints.": "checkpoint_url",
    "Are national parks closed in this community?": "parks"
})
full_policies.pop("If you found any additional information that doesn't fit into the above categories, please describe it here.")
full_policies.pop("Please provide a URL for the information above.")

# Fuzzy mapping function to correctly map the census AIANNHCE codes to user-entered names
def fuzzymap(user,dictum):
  commons = ["New","North", "South", "Dakota", "SDTSA", "OTSA", "ANVSA", "Big", "Native", "Community", "Tribe", "Pueblo", "Village", "American", "Mexico", "Nation", "Native", "and", "of", "Indians", "Indian", "Tribe", "Tribes" "Off", "-", "Reservation", "Trust", "Land" "The", "the", ",", "(", ")"]
  matches = []
  for i, query in enumerate(user):
    # delete common words from the test queries
    print('query number:' + str(i))
    print('query is: ' + query)
    # time.sleep(0.25)

    # hardcoding exceptions that fuzzymatch does not catch
    ex_dict = {
        'Pueblo of Sandia': 'Sandia Pueblo',
        'Sandia Pueblo': 'Sandia Pueblo',
        'Pueblo of Taos': 'Taos Pueblo and Off-Reservation Trust Land',
        'Confederated Tribes and Bands of the Yakama Nation': 'Yakama Nation Reservation and Off-Reservation Trust Land',
        'Kiowa Indian Tribe of Oklahoma': 'Kiowa-Comanche-Apache-Fort Sill Apache OTSA',
        'Mashpee Wampanoag Tribe':'Wampanoag-Aquinnah Trust Land',
        'The Osage Nation': 'Osage Reservation',
        'Wrangell Cooperative Association': 'Wrangell ANVSA',
        'Colorado River Indian Tribes of the Colorado River Indian Reservation (Arizona and California)': 'Colorado River Indian Reservation',
        'Four Winds Tribe Louisiana Cherokee Confederacy': 'Four Winds Cherokee SDTSA',
        'Makah Indian Tribe of the Makah Indian Reservation':  'Makah Indian Reservation',
        'Hualapai Indian Tribe of the Hualapai Indian Reservation': 'Hualapai Indian Reservation and Off-Reservation Trust Land',
        'Reno-Sparks Indian Colony': 'Reno-Sparks Indian Colony',
        'Turtle Mountain Band of Chippewa Indians of North Dakota': 'Turtle Mountain Reservation and Off-Reservation Trust Land',
        'Yankton Sioux Tribe of South Dakota': 'Yankton Reservation',
        'Fort Belknap Indian Community of the Fort Belknap Reservation of Montana': 'Fort Belknap Reservation and Off-Reservation Trust Land',
        'Wyandotte Nation': 'Wyandotte OTSA',
        'Cherokee Nation': 'Cherokee OTSA',
        'Mescalero Apache Tribe of the Mescalero Reservation': 'Mescalero Reservation',
        'Pueblo of Nambe': 'Nambe Pueblo and Off-Reservation Trust Land',
        'Pueblo of Tesuque': 'Tesuque Pueblo and Off-Reservation Trust Land',
        'Pueblo of Acoma': 'Acoma Pueblo and Off-Reservation Trust Land',
        'Mechoopda Indian Tribe of Chico Rancheria': 'Mechoopda TDSA',
        'Bad River Band of the Lake Superior Tribe of Chippewa Indians of the Bad River Reservation': 'Bad River Reservation',
        'Ute Mountain Ute Tribe (Colorado, New Mexico and Utah)': 'Ute Mountain Reservation and Off-Reservation Trust Land',
        'Ute Mountain Tribe of the Ute Mountain Reservation (Colorado, New Mexico and Utah)': 'Ute Mountain Reservation and Off-Reservation Trust Land',
        'Confederated Salish and Kootenai Tribes of the Flathead Reservation': 'Flathead Reservation',
        'Cherokee Tribe of Northeast Alabama': 'Cherokee Tribe of Northeast Alabama SDTSA',
        

    }
    if query in ex_dict.keys():
      bestmatch = ex_dict[query]
      thresh = 10000
    else:
        for comm in commons:
          query = query.replace(comm, "")
        
        thresh = 0
        for key in dictum.keys():
          for comm in commons:
            key2 = key.replace(comm, "")
          score = fuzz.partial_ratio(query, key2)
          if score > thresh:
            thresh = score
            bestmatch = key
      
    # user2[i] = bestmatch
    matches.append(bestmatch)
    print('bestmatch is: ' + bestmatch)
    print('fuzzy score is: ' + str(thresh))


  user2 = pd.DataFrame({'header': matches}) 
  codes = user2.header.map(dictum)

  return codes


In [ ]:
# full_policies["AIANNHCE"] = full_policies.community_name.map(aian_code_mapping)
full_policies["AIANNHCE"] = fuzzymap(full_policies.community_name,aian_code_mapping)
full_policies.set_index("AIANNHCE", inplace=True)
full_policies["updated"] = pd.to_datetime(full_policies.pop("Timestamp"))
for col in dimensions:
  full_policies[col] = full_policies[col].map({"Yes": True, "No": False, "N/A":float('nan'), "":float('nan')})

full_policies['fdpir'] = full_policies['fdpir'].map({"Yes": True, "No": False, "N/A":float('nan'), "":float('nan')})
full_policies['parks'] = full_policies['parks'].map({"Yes": True, "No": False, "N/A":float('nan'), "":float('nan')})

# Convert timestamp from ambiguous DD/MM/YYYY to ISO 8601 friendly YYYY-MM-DD
for col in dimensions:
  full_policies[col + "_date"] = pd.to_datetime(full_policies[col + "_date"]).dt.strftime("%Y-%m-%d")

query number:0
query is: Pueblo of Sandia
bestmatch is: Sandia Pueblo
fuzzy score is: 10000
query number:1
query is: Cherokee Nation
bestmatch is: Cherokee OTSA
fuzzy score is: 10000
query number:2
query is: Standing Rock Sioux Tribe (North Dakota and South Dakota)
bestmatch is: Standing Rock Reservation
fuzzy score is: 50
query number:3
query is: The Choctaw Nation of Oklahoma
bestmatch is: Choctaw OTSA
fuzzy score is: 75
query number:4
query is: Wind River Inter-Tribal 
bestmatch is: Wind River Reservation and Off-Reservation Trust Land
fuzzy score is: 65
query number:5
query is: Puyallup Tribe of the Puyallup Reservation
bestmatch is: Puyallup Reservation and Off-Reservation Trust Land
fuzzy score is: 52
query number:6
query is: Confederated Tribes and Bands of the Yakama Nation
bestmatch is: Yakama Nation Reservation and Off-Reservation Trust Land
fuzzy score is: 10000
query number:7
query is: Navajo Nation (Arizona, New Mexico and Utah)
bestmatch is: Navajo Nation Reservation and 

In [ ]:
full_policies.school.unique()

array([nan, True], dtype=object)

### Deduplicate rows.

Naive assumption - we assume that restrictions are never lifted, so the heaviest restriction reported for fips is still active.

We will likely have to revisit this in the future.

In [ ]:
# No deduplication currently necessary
policies_dedup = full_policies
policies_dedup = policies_dedup.reset_index().drop_duplicates(subset="AIANNHCE")

# # Remove if NA for FIPS (if submitter of data changed county name and FIPS could not be correctly mapped
# policies_min = policies_dedup[~policies_dedup.fips.isna() & (policies_dedup.fips < 100000) & (policies_dedup.fips >= 0)].copy()

policies_min = policies_dedup.set_index('AIANNHCE')
# policies_min2 = policies_min

In [ ]:
# policies_min2.set_index('AIANNHCE', inplace=True);
# full_policies.groupby('fips')['updated'].sum()

In [ ]:
# function that has the following aggregation behavior:
# pick the most recently submitted entry on any particular policy, inherit latest policy for each policy 
# EXCEPT if 2 or more within 24 hours of the last entered policy
# in the case of 2 entries with disagreement, pick the one that is most open (i.e. favor No over Yes)
# if 3+ entries within 24 hours then take majority vote, leaning No in case of tie

def agg24(df):
  polcols = dimensions
  # one entry should be returned for each county, initialize to first entry and edit later
  findf = df.head(1)
  for col in polcols:
    # find last non-null datapoint for that policy
    filldf = df[~df[col].isnull()]
    tmax = filldf.updated.max()
    tcut = tmax - np.timedelta64(24,'h')
    reduced = filldf.drop(filldf[filldf.updated < tcut].index)

    # will return date and URL of the last entry that matched the final boolean
    urlcol = col + '_url'
    datecol = col + '_date'
    
    if reduced[col].size == 0:
       decision = float('nan')
    elif np.sum(reduced[col])/reduced[col].size > 0.50001:
      decision = True
    else:
      decision = False
    findf[col].iloc[0] = decision
    sourced = reduced[reduced[col] == decision]

    if sourced[col].shape[0] > 0:
      # print(sourced.shape[0])
      findf[urlcol].iloc[0] = sourced[urlcol].iloc[sourced.shape[0]-1]
      findf[datecol].iloc[0] = sourced[datecol].iloc[sourced.shape[0]-1]

 #   if reduced[col].isna().all()
  return findf
  

In [ ]:
# No aggregation needed for now, doing simple deduplication
# policies_agg = full_policies.groupby('fips').apply(agg24)
# policies_agg = policies_agg.droplevel(1)

In [ ]:
# policies_agg.pop("county_name");
# policies_min.set_index('fips', inplace=True);
#Identifying the problematic row
# policies_debug = policies_min[0:140]
# policies_min = policies_debug

In [ ]:
policies_min.shape

(126, 34)

In [ ]:
policies_min.head(100)

,community_name,population,testing,testing_date,testing_url,school,school_date,school_url,work,work_date,work_url,shelter,shelter_date,shelter_url,shelter_enforcement,shelter_enforcement_date,shelter_enforcement_url,event,event_date,event_url,transport,transport_date,transport_url,fdpir,fdpir_url,revenue,revenue_date,revenue_url,checkpoint,checkpoint_date,checkpoint_url,testing_time,parks,updated
AIANNHCE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3370,Pueblo of Sandia,417,NaN,NaN,,NaN,NaN,,True,2020-03-17,https://www.sandiacasino.com/your-safety-matters/,NaN,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,,NaN,,NaN,NaN,,NaN,NaN,,,NaN,2020-05-09 16:36:38
5550,Cherokee Nation,,True,2020-04-02,https://www.theonefeather.com/2020/04/cherokee...,True,2020-03-17,https://www.tahlequahdailypress.com/news/chero...,True,2020-03-17,https://www.tahlequahdailypress.com/news/chero...,NaN,NaN,,NaN,NaN,,True,2020-03-17,https://www.tahlequahdailypress.com/news/chero...,NaN,NaN,,NaN,,NaN,NaN,,NaN,NaN,,,NaN,2020-05-09 16:37:20
3970,Standing Rock Sioux Tribe (North Dakota and So...,8618,True,2020-05-09,https://www.kfyrtv.com/content/news/Standing-R...,True,2020-03-30,https://www.standing-rock.k12.nd.us/,True,2020-03-13,https://www.srstcovid19.com/,True,2020-03-13,https://www.srstcovid19.com/,False,NaN,,True,NaN,https://www.standingrock.org/content/covid-19,NaN,NaN,,NaN,,NaN,NaN,,NaN,NaN,,,NaN,2020-05-09 16:43:16
5590,The Choctaw Nation of Oklahoma,79006,True,2020-04-20,https://www.choctawnation.com/news-events/pres...,True,2020-03-25,https://www.choctawnation.com/news-events/pres...,True,2020-03-17,http://choctawnation.com/covid-19,False,NaN,,False,2020-01-01,,True,2020-03-17,http://choctawnation.com/covid-19,NaN,NaN,,NaN,,NaN,NaN,,NaN,NaN,,,NaN,2020-05-09 16:49:26
4610,Wind River Inter-Tribal,26490,True,2020-04-06,https://www.facebook.com/WRFCHCOfficial/posts/...,True,NaN,https://www.wyomingpublicmedia.org/post/wind-r...,True,NaN,https://county10.com/wp-content/uploads/2020/0...,True,2020-03-23,https://www.wyomingpublicmedia.org/post/covid-...,True,2020-04-18,https://trib.com/news/state-and-regional/triba...,True,2020-03-31,https://county10.com/wp-content/uploads/2020/0...,NaN,NaN,,NaN,,NaN,NaN,,NaN,NaN,,,NaN,2020-05-09 16:49:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9838,Occaneechi Band of teh Saponi Nation,9522,True,2020-05-14,https://www.wfmynews2.com/article/news/health/...,NaN,NaN,http://obsn.org/,NaN,NaN,http://obsn.org/,NaN,NaN,http://obsn.org/,NaN,NaN,http://obsn.org/,NaN,NaN,http://obsn.org,NaN,NaN,http://obsn.org/,False,http://obsn.org/,NaN,NaN,http://obsn.org/,False,NaN,http://obsn.org/,20,NaN,2020-05-30 14:52:16
9745,Haliwa-Saponi Indian Tribe,4300,True,2020-05-11,https://www.fastmed.com/urgent-care-centers/ro...,True,2020-03-16,https://www.facebook.com/photo?fbid=1093763964...,True,2020-03-30,https://www.facebook.com/photo?fbid=2747993951...,True,2020-03-30,https://www.facebook.com/photo?fbid=2747993951...,False,NaN,,True,2020-03-17,https://www.facebook.com/photo?fbid=2717200028...,NaN,NaN,,True,https://www.facebook.com/haliwasaponitribalsch...,True,2020-03-30,https://www.facebook.com/photo?fbid=2747993951...,NaN,NaN,,27,False,2020-05-30 17:22:00
305,Blackfeet Tribe of the Blackfeet Indian Reserv...,17321,True,NaN,http://www.cutbankpioneerpress.com/glacier_rep...,True,2020-04-20,https://www.bps.k12.mt.us/school_closure__covi...,True,2020-04-27,https://blackfeetnation.com/wp-content/uploads...,True,2020-04-01,https://blackfeetnation.com/wp-content/uploads...,True,2020-04-01,https://blackfeetnation.com/wp-content/uploads...,True,2020-04-08,https://blackfeetnation.com/wp-content/uploads...,False,NaN,,True,https://www.fastblackfeet.org/resources/blackf...,True,2020-04-27,https://blackfeetnation.com/wp-content/uploads...,True,2020-04-01,https://blackfeetnation.com/wp-content/uploads...,17,True,2020-06-11 13:08:00


In [ ]:
# names only match between user input and Census for 6 counties automatically, now all 96 counties

len(full_policies.index.unique())

126

In [ ]:
len(full_policies.reset_index().AIANNHCE.unique())

126

In [ ]:
full_policies

,community_name,population,testing,testing_date,testing_url,school,school_date,school_url,work,work_date,work_url,shelter,shelter_date,shelter_url,shelter_enforcement,shelter_enforcement_date,shelter_enforcement_url,event,event_date,event_url,transport,transport_date,transport_url,fdpir,fdpir_url,revenue,revenue_date,revenue_url,checkpoint,checkpoint_date,checkpoint_url,testing_time,parks,updated
AIANNHCE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3370,Pueblo of Sandia,417,NaN,NaN,,NaN,NaN,,True,2020-03-17,https://www.sandiacasino.com/your-safety-matters/,NaN,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,,NaN,,NaN,NaN,,NaN,NaN,,,NaN,2020-05-09 16:36:38
5550,Cherokee Nation,,True,2020-04-02,https://www.theonefeather.com/2020/04/cherokee...,True,2020-03-17,https://www.tahlequahdailypress.com/news/chero...,True,2020-03-17,https://www.tahlequahdailypress.com/news/chero...,NaN,NaN,,NaN,NaN,,True,2020-03-17,https://www.tahlequahdailypress.com/news/chero...,NaN,NaN,,NaN,,NaN,NaN,,NaN,NaN,,,NaN,2020-05-09 16:37:20
3970,Standing Rock Sioux Tribe (North Dakota and So...,8618,True,2020-05-09,https://www.kfyrtv.com/content/news/Standing-R...,True,2020-03-30,https://www.standing-rock.k12.nd.us/,True,2020-03-13,https://www.srstcovid19.com/,True,2020-03-13,https://www.srstcovid19.com/,False,NaN,,True,NaN,https://www.standingrock.org/content/covid-19,NaN,NaN,,NaN,,NaN,NaN,,NaN,NaN,,,NaN,2020-05-09 16:43:16
5590,The Choctaw Nation of Oklahoma,79006,True,2020-04-20,https://www.choctawnation.com/news-events/pres...,True,2020-03-25,https://www.choctawnation.com/news-events/pres...,True,2020-03-17,http://choctawnation.com/covid-19,False,NaN,,False,2020-01-01,,True,2020-03-17,http://choctawnation.com/covid-19,NaN,NaN,,NaN,,NaN,NaN,,NaN,NaN,,,NaN,2020-05-09 16:49:26
4610,Wind River Inter-Tribal,26490,True,2020-04-06,https://www.facebook.com/WRFCHCOfficial/posts/...,True,NaN,https://www.wyomingpublicmedia.org/post/wind-r...,True,NaN,https://county10.com/wp-content/uploads/2020/0...,True,2020-03-23,https://www.wyomingpublicmedia.org/post/covid-...,True,2020-04-18,https://trib.com/news/state-and-regional/triba...,True,2020-03-31,https://county10.com/wp-content/uploads/2020/0...,NaN,NaN,,NaN,,NaN,NaN,,NaN,NaN,,,NaN,2020-05-09 16:49:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,Ho-Chunk Nation of Wisconsin,1632,True,2020-05-11,https://www.wxpr.org/post/evers-announces-free...,NaN,NaN,,True,2020-03-13,https://ho-chunknation.com/wp-content/uploads/...,True,2020-03-20,https://ho-chunknation.com/wp-content/uploads/...,False,NaN,,True,2020-03-20,https://ho-chunknation.com/wp-content/uploads/...,False,NaN,,True,http://health.ho-chunk.com/fd.html,True,2020-03-20,https://ho-chunknation.com/wp-content/uploads/...,False,NaN,,1,NaN,2020-06-28 23:43:20
1450,Ho-Chunk Nation of Wisconsin,,NaN,NaN,,NaN,NaN,,False,2020-05-15,https://ho-chunknation.com/wp-content/uploads/...,NaN,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,,NaN,,False,2020-05-27,https://ho-chunknation.com/wp-content/uploads/...,NaN,NaN,,,NaN,2020-06-28 23:45:54
2055,Lower Brule Sioux Tribe of the Lower Brule Res...,1631,False,NaN,,True,2020-03-31,https://www.lowerbruleschools.org/pages/upload...,False,NaN,,True,2020-04-06,https://88f41f86-257c-403a-9c6f-33afd6701b84.f...,False,NaN,,True,2020-04-06,https://88f41f86-257c-403a-9c6f-33afd6701b84.f...,False,NaN,,True,https://www.fns.usda.gov/contacts?f%5B0%5D=pro...,True,2020-04-12,http://www.thegoldenbuffalocasino.com/announce...,True,2020-04-30,https://88f41f86-257c-403a-9c6f-33afd6701b84.f...,,True,2020-06-28 23:51:03


In [ ]:
with open("all_community_policies.csv", "w") as outfile:
  outfile.write(full_policies.to_csv())

In [ ]:
with open("community_policies.json", "w") as outfile:
  outfile.write(policies_min.to_json(orient="index"))

In [ ]:
with open("community_policies.csv", "w") as outfile:
  outfile.write(policies_min.to_csv())

In [ ]:
# Reformatting for Tableau
# policies_min2['county_only'] = policies_min2['county_name'].str.split(', ', expand=True)[0]
# policies_min2['state_only'] = policies_min2['county_name'].str.split(', ', expand=True)[1]

In [ ]:
# policies_min2.head()

In [ ]:
# policies_min.columns

In [ ]:
# cols = ['county_only','state_only']
# cols = [*cols, *policies_min.columns]

In [ ]:
# policies_min2 = policies_min2[cols]

In [ ]:
# policies_min2.head()

In [ ]:
# with open("county_policies_2.csv", "w") as outfile:
  # outfile.write(policies_min2.to_csv())